In [1]:
import sqlite3 as lite
import datetime as dt
import sys
import pandas as pd

In [2]:
# %load 'helpers.py'
# helper functions

import adafruit_dht
import board
import time
import psutil
import sqlite3 as lite
import sys
import os

def init_dht(model=22):
    model = int(model)
    print(model)
    flag = True
    while flag:
        try:
            print("attempting to initialize device...")
            if model == int(22):
                print("DHT22...")
                dhtDevice = adafruit_dht.DHT22(board.D4)
            elif model == int(11):
                print("DHT11...")
                dhtDevice = adafruit_dht.DHT11(board.D4)
            else:
                print("incorrect DHT model number--enter '11' or '22'")
                return None
            dhtDevice.temperature
            print("successfully initialized DHT" + str(model))

        except RuntimeError as error:
            print(error.args[0])
            print("failed to initialize, cleaning up libgpio processes...")
            for p in psutil.process_iter():
                if p.name()[:8] == 'libgpiod':
                    print("killing {} (pid: {})".format(p.name(), p.pid))
                    p.kill()
            time.sleep(5)
            continue
        flag = False
    return dhtDevice


def read_dht(dhtDevice, deg='F'):
    while True:
        try:
            t = dhtDevice.temperature
            h = dhtDevice.humidity
            if deg == 'F':
                t = t * (9 / 5) + 32
            return (t, h)
        except RuntimeError as error:
            print(error.args[0])
            time.sleep(2)


def create_table(location, debug=False):
    # Create DHT_data table if it doesn't exist already
    if not os.path.isdir(location):
        assert False, "cannot create table, directory given (" + location + ") does not exist!"
    con = lite.connect(os.path.join(location, 'sensorsData.db'))
    with con:
        cur = con.cursor()
        #if table 'DHT_data' exists:
        try:
            cur.execute("CREATE TABLE DHT_data(timestamp DATETIME, temp NUMERIC, hum NUMERIC)")
            if debug:
                print("Created DHT_data table")
        except:
            if debug:
                print("DHT_data table already exists")
    return con


def add_entry(db, timestamp, temp, hum, debug=False):
    # Add a temp/humidity reading to the database
    with db as con:
        cur = con.cursor()
        if debug:
            print("Adding DHT reading to DHT_data table: ", timestamp, temp, hum)
        cur.execute("INSERT INTO DHT_data VALUES((?), (?), (?))", (timestamp, temp, hum))

In [19]:
db_dir = '/home/pi/repos/humidity_logging/db'
db = create_table(db_dir, debug=True)

DHT_data table already exists


In [4]:
DHT = init_dht(22)

22
attempting to initialize device...
DHT22...
successfully initialized DHT22


In [5]:
t, h = read_dht(DHT)
now = dt.datetime.now()
add_entry(db, now, t, h, debug=True)

Adding DHT reading to DHT_data table:  2020-12-13 12:23:54.209799 66.56 59.6


In [6]:
help(db)

Help on Connection object:

class Connection(builtins.object)
 |  SQLite database connection object.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, /, *args, **kwargs)
 |      Call self as a function.
 |  
 |  __enter__(...)
 |      For context manager. Non-standard.
 |  
 |  __exit__(...)
 |      For context manager. Non-standard.
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  backup(...)
 |      Makes a backup of the database. Non-standard.
 |  
 |  close(...)
 |      Closes the connection.
 |  
 |  commit(...)
 |      Commit the current transaction.
 |  
 |  create_aggregate(...)
 |      Creates a new aggregate. Non-standard.
 |  
 |  create_collation(...)
 |      Creates a collation function. Non-standard.
 |  
 |  create_function(...)
 |      Creates a new function. Non-standard.
 |  
 |  cursor(...)
 |      Return a cursor for the connection.
 |  
 |  enable_load_extension(...)
 |      Enable dyn

In [20]:
it = db.iterdump()

for i in it:
    print(i)

BEGIN TRANSACTION;
CREATE TABLE DHT_data(timestamp DATETIME, temp NUMERIC, hum NUMERIC);
INSERT INTO "DHT_data" VALUES('2020-12-11 23:42:34.996891',62.96,61.9);
INSERT INTO "DHT_data" VALUES('2020-12-11 23:43:35.402283',62.96,62);
INSERT INTO "DHT_data" VALUES('2020-12-13 10:50:16.520659',66.2,61.1);
INSERT INTO "DHT_data" VALUES('2020-12-13 10:50:54.251205',66.2,60.9);
INSERT INTO "DHT_data" VALUES('2020-12-13 10:51:57.014126',66.2,60.8);
INSERT INTO "DHT_data" VALUES('2020-12-13 10:57:31.549155',66.2,60.8);
INSERT INTO "DHT_data" VALUES('2020-12-13 10:58:38.935832',66.2,60.6);
INSERT INTO "DHT_data" VALUES('2020-12-13 11:00:59.412823',66.2,60.6);
INSERT INTO "DHT_data" VALUES('2020-12-13 11:02:57.964563',66.2,60.9);
COMMIT;


In [21]:
df = pd.read_sql_query("SELECT * from DHT_data", db)

In [22]:
df['hum'].values

array([61.9, 62. , 61.1, 60.9, 60.8, 60.8, 60.6, 60.6, 60.9])